In [14]:
import pandas as pd
import numpy as np

import yfinance as yf

In [15]:
TICKS = [
    'SPY',
    'EFA',
    'EEM',
    'PSP',
    'QAI',
    'HYG',
    'DBC',
    'IYR',
    'IEF',
    'BWX',
    'TIP',
    'SHV',
]

FLDS = ['shortName','quoteType','currency','volume','totalAssets','longBusinessSummary']

In [16]:
info = pd.DataFrame(index=TICKS,columns=FLDS)
info.index.name = 'ticker'
for tick in info.index:
    temp = yf.Ticker(tick).get_info()

    for fld in FLDS:
        if fld in temp.keys():
            info.loc[tick,fld] = temp[fld]

In [17]:
info

,shortName,quoteType,currency,volume,totalAssets,longBusinessSummary
ticker,,,,,,
SPY,SPDR S&P 500,ETF,USD,25604208,627773800448,The Trust seeks to achieve its investment obje...
EFA,iShares MSCI EAFE ETF,ETF,USD,10653257,54985711616,The fund generally will invest at least 80% of...
EEM,iShares MSCI Emerging Index Fun,ETF,USD,17962107,17468592128,The fund generally will invest at least 80% of...
PSP,Invesco Global Listed Private E,ETF,USD,8928,277930496,The fund generally will invest at least 90% of...
QAI,NYLI Hedge Multi-Strategy Track,ETF,USD,49257,637390272,"The fund is a ""fund of funds"" which means it i..."
HYG,iShares iBoxx $ High Yield Corp,ETF,USD,22374708,15881510912,The underlying index is a rules-based index co...
DBC,Invesco DB Commodity Index Trac,ETF,USD,478168,1387142912,The fund pursues its investment objective by i...
IYR,iShares U.S. Real Estate ETF,ETF,USD,2699001,4990495744,The fund seeks to track the investment results...
IEF,iShares 7-10 Year Treasury Bond,ETF,USD,2340833,32854654976,The underlying index measures the performance ...


In [18]:
STARTDATE = '2015-01-01'
ENDDATE = '2024-12-31'

tickers = list(info.index.values)
df = yf.download(tickers, start=STARTDATE, end=ENDDATE, auto_adjust=False)['Adj Close']

[*********************100%***********************]  12 of 12 completed


In [19]:
prices = df.resample('M').last()

# eliminate timezones
prices.index = prices.index.tz_localize(None)

rets = prices.pct_change().dropna()

# change to excess returns, in excess of short-term treasury
retsx = rets.subtract(rets['SHV'], axis=0)
retsx = retsx.drop(columns=['SHV'])

/var/folders/zx/3v_qt0957xzg3nqtnkv007d00000gn/T/ipykernel_12293/3831157974.py:1: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  prices = df.resample('M').last()


In [20]:
rets

Ticker,BWX,DBC,EEM,EFA,HYG,IEF,IYR,PSP,QAI,SHV,SPY,TIP
Date,,,,,,,,,,,,
2015-02-28,-0.010834,0.044253,0.044080,0.063378,0.022312,-0.024717,-0.025976,0.064102,0.034138,0.000181,0.056204,-0.012886
2015-03-31,-0.013922,-0.060539,-0.014973,-0.014286,-0.009478,0.008561,0.010745,-0.010454,-0.001667,-0.000091,-0.015706,-0.004819
2015-04-30,0.019579,0.071471,0.068527,0.036466,0.008714,-0.006330,-0.048160,0.053982,0.002004,0.000091,0.009834,0.006779
2015-05-31,-0.032312,-0.031711,-0.041045,0.001955,0.003555,-0.004164,-0.003311,0.026868,-0.000333,0.000000,0.012856,-0.010056
2015-06-30,-0.007442,0.016375,-0.029309,-0.031182,-0.018871,-0.016309,-0.043979,-0.019153,-0.013671,0.000091,-0.020312,-0.010246
...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-31,0.030519,-0.020815,0.009779,0.032603,0.015474,0.013458,0.054008,0.001225,0.007648,0.004979,0.023366,0.007990
2024-09-30,0.023484,0.007237,0.057413,0.007833,0.016971,0.013825,0.030631,0.051617,0.014548,0.004586,0.021005,0.014976
2024-10-31,-0.048497,0.014369,-0.030746,-0.052732,-0.009636,-0.033874,-0.034947,-0.013779,-0.005923,0.003576,-0.008924,-0.018469


In [21]:
retsx

Ticker,BWX,DBC,EEM,EFA,HYG,IEF,IYR,PSP,QAI,SPY,TIP
Date,,,,,,,,,,,
2015-02-28,-0.011015,0.044071,0.043898,0.063196,0.022130,-0.024898,-0.026157,0.063921,0.033956,0.056023,-0.013067
2015-03-31,-0.013831,-0.060448,-0.014882,-0.014195,-0.009387,0.008652,0.010836,-0.010363,-0.001576,-0.015615,-0.004728
2015-04-30,0.019488,0.071380,0.068436,0.036375,0.008623,-0.006421,-0.048251,0.053891,0.001913,0.009743,0.006688
2015-05-31,-0.032312,-0.031711,-0.041045,0.001955,0.003555,-0.004164,-0.003311,0.026868,-0.000333,0.012856,-0.010056
2015-06-30,-0.007532,0.016284,-0.029399,-0.031273,-0.018962,-0.016400,-0.044070,-0.019244,-0.013762,-0.020402,-0.010337
...,...,...,...,...,...,...,...,...,...,...,...
2024-08-31,0.025540,-0.025794,0.004799,0.027624,0.010494,0.008478,0.049029,-0.003754,0.002669,0.018386,0.003011
2024-09-30,0.018898,0.002651,0.052827,0.003247,0.012385,0.009239,0.026045,0.047032,0.009962,0.016419,0.010390
2024-10-31,-0.052073,0.010793,-0.034322,-0.056308,-0.013212,-0.037450,-0.038523,-0.017355,-0.009499,-0.012500,-0.022045


In [22]:
with pd.ExcelWriter('../data/multi_asset_etf_data_2025.xlsx') as writer:  
    info.to_excel(writer, sheet_name= 'descriptions')
    prices.to_excel(writer, sheet_name= 'prices')
    rets.to_excel(writer, sheet_name='total returns')
    retsx.to_excel(writer, sheet_name='excess returns')